In [1]:
import numpy as np
import pandas as pd
from gensim.models.word2vec import Word2Vec
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Conv1D, MaxPool1D, GlobalMaxPool1D, Activation
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
import re
import nltk
from nltk.corpus import stopwords
from sklearn import preprocessing
from gensim.parsing.preprocessing import remove_stopwords
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler


Using TensorFlow backend.


In [2]:
df = pd.read_csv("3000tweet/3000tweet.csv", sep=';',encoding="utf8")

In [3]:
df.head()

,tweet,durum
0,dun Turkcelle tepkilerimizden sonra bugün Turk...,1
1,girmezmiyim.. Turkcell kartim bile var.. Yarin...,1
2,tam tünelden gecerken 3g cekiyordu :D türkcell...,1
3,turkcell superonline fiber internet veya ADSL ...,1
4,bence Gnçtrkcll Ark Winterfest 2012'de 1.olur ...,1


In [4]:
df.isna().sum()

tweet    1
durum    0
dtype: int64

In [5]:
#df.dropna(inplace=True)
df = df.dropna().reset_index(drop=True)
df.isnull().sum()

tweet    0
durum    0
dtype: int64

In [6]:
df.isna().sum()

tweet    0
durum    0
dtype: int64

In [7]:
def preprocess_text(sen):
    
    #Remove numeric
    sentence = re.sub('[\d\s]', ' ', str(sen))

    # Remove punctuations and numbers
    sentence = re.sub('[^\w\s]', ' ', str(sentence))
    
    #Single character removal
    sentence = re.sub(r"\b[\w\s]\b", ' ',str(sentence))
    
    #Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)
    
    stop_word_list = stopwords.words('turkish')
    
    tokens = WPT.tokenize(sentence)
    filtered_tokens = [token for token in tokens if token not in stop_word_list]
    single_doc = ' '.join(filtered_tokens)

    return single_doc.lower()


In [8]:
WPT = nltk.WordPunctTokenizer()
df['tweet'] = df['tweet'].apply(preprocess_text)

In [9]:
df[:10]

,tweet,durum
0,dun turkcelle tepkilerimizden sonra bugün turk...,1
1,girmezmiyim turkcell kartim bile var yarindan ...,1
2,tam tünelden gecerken cekiyordu türkcell in re...,1
3,turkcell superonline fiber internet adsl sabit...,1
4,bence gnçtrkcll ark winterfest olur gelir,1
5,turkcell sana kıyak geçiyor,1
6,turkcelle bağlan hayata diyorum,1
7,kimse takmıyo beni yaaaaa turkcell bana mesaj ...,1
8,bu turkcell pusula uygulaması iphone kullanıcı...,1
9,zaman sorun yok bende turkcell iyi bir tarife ...,1


In [10]:
mes = []
for i in df['tweet']:
    mes.append(i.split())
print(mes[:2])

[['dun', 'turkcelle', 'tepkilerimizden', 'sonra', 'bugün', 'turkcell', 'twittera', 'sponsor', 'olmuş', 'ne', 'tesadüf', 'değil'], ['girmezmiyim', 'turkcell', 'kartim', 'bile', 'var', 'yarindan', 'itibaren', 'onu', 'takicam', 'twittersiz', 'denedim', 'sadece', 'gun', 'yapabildim']]


In [11]:
word2vec_model = Word2Vec(mes, size=500, window=3, min_count=1, workers=16)
print(word2vec_model)

Word2Vec(vocab=11748, size=500, alpha=0.025)


In [12]:
token = Tokenizer(11748)
token.fit_on_texts(df['tweet'])
text = token.texts_to_sequences(df['tweet'])
text = pad_sequences(text, 25)
print(text[:2])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0  614
   389 3159   29   33    1 3160  916  155   61 1772   36]
 [   0    0    0    0    0    0    0    0    0    0    0 3161    1 3162
    32    9 3163 1773  197 3164 3165  749   43  168 3166]]


In [13]:
scaler = StandardScaler()
text = scaler.fit_transform(text)
print(text[:5])

[[ 0.          0.         -0.01831695 -0.02298629 -0.03349231 -0.04242505
  -0.07520293 -0.11601734 -0.16286415 -0.22058396 -0.27752467 -0.31900664
  -0.36151624 -0.16407776 -0.31710276  0.65779244 -0.50410186 -0.54242122
  -0.59707035  0.37242237 -0.3848634  -0.6506629  -0.69952181 -0.3074559
  -0.83593737]
 [ 0.          0.         -0.01831695 -0.02298629 -0.03349231 -0.04242505
  -0.07520293 -0.11601734 -0.16286415 -0.22058396 -0.27752467  1.22098065
  -0.36106531  0.8687639  -0.45207122 -0.47727483  0.61394277  0.0463323
  -0.5334018   0.37368829  0.30229872 -0.47034052 -0.70477794 -0.75456744
   0.0350108 ]
 [ 0.          0.         -0.01831695 -0.02298629 -0.03349231 -0.04242505
  -0.07520293 -0.11601734 -0.16286415 -0.22058396 -0.27752467 -0.31900664
  -0.36151624 -0.41296503 -0.41199655  0.66067515  0.61572651  0.51868857
  -0.58894936 -0.62449053 -0.12270907 -0.69134169  0.20832834  0.08251234
  -0.35204636]
 [ 0.          0.         -0.01831695 -0.02298629 -0.03349231 -0.0424

In [14]:
le = preprocessing.LabelEncoder()
y = le.fit_transform(df['durum'])
y = to_categorical(y)

In [15]:
print(y)

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [16]:
x_train, x_test, y_train, y_test = train_test_split(np.array(text), y, test_size=0.2, stratify=y)

In [17]:
keras_model = Sequential()
keras_model.add(word2vec_model.wv.get_keras_embedding(True))
keras_model.add(Dropout(0.2))
keras_model.add(Conv1D(50, 3, activation='relu', padding='same', strides=1))
keras_model.add(Conv1D(50, 3, activation='relu', padding='same', strides=1))
keras_model.add(MaxPool1D())
keras_model.add(Dropout(0.2))
keras_model.add(Conv1D(100, 3, activation='relu', padding='same', strides=1))
keras_model.add(Conv1D(100, 3, activation='relu', padding='same', strides=1))
keras_model.add(MaxPool1D())
keras_model.add(Dropout(0.2))
keras_model.add(Conv1D(200, 3, activation='relu', padding='same', strides=1))
keras_model.add(Conv1D(200, 3, activation='relu', padding='same', strides=1))
keras_model.add(GlobalMaxPool1D())
keras_model.add(Dropout(0.2))
keras_model.add(Dense(200))
keras_model.add(Activation('relu'))
keras_model.add(Dropout(0.2))
keras_model.add(Dense(3))
keras_model.add(Activation('softmax'))
keras_model.compile(loss='binary_crossentropy', metrics=['acc'], optimizer='adam')
history = keras_model.fit(x_train, y_train, batch_size=16, epochs=10, validation_data=(x_test, y_test))

D:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 2399 samples, validate on 600 samples
Epoch 1/10
2399/2399 [==============================] - 15s 6ms/step - loss: 0.5048 - acc: 0.7521 - val_loss: 0.3997 - val_acc: 0.8094
Epoch 2/10
2399/2399 [==============================] - 14s 6ms/step - loss: 0.3084 - acc: 0.8608 - val_loss: 0.2782 - val_acc: 0.8644
Epoch 3/10
2399/2399 [==============================] - 14s 6ms/step - loss: 0.2890 - acc: 0.8641 - val_loss: 0.2832 - val_acc: 0.8633
Epoch 4/10
2399/2399 [==============================] - 14s 6ms/step - loss: 0.2886 - acc: 0.8662 - val_loss: 0.2928 - val_acc: 0.8661
Epoch 5/10
2399/2399 [==============================] - 14s 6ms/step - loss: 0.2783 - acc: 0.8731 - val_loss: 0.2635 - val_acc: 0.8761
Epoch 6/10
2399/2399 [==============================] - 14s 6ms/step - loss: 0.2804 - acc: 0.8720 - val_loss: 0.2637 - val_acc: 0.8767
Epoch 7/10
2399/2399 [==============================] - 13s 6ms/step - loss: 0.2736 - acc: 0.8741 - val_loss: 0.2703 - val_acc: 0.8706
Epoch 8/

KeyboardInterrupt: 

In [ ]:
keras_model.summary()

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(12,5))
ax[0].plot(history.history['loss'], color='b', label="Training loss")
ax[0].plot(history.history['val_loss'], color='r', label="Validation loss",axes =ax[0])
ax[0].grid(color='black', linestyle='-', linewidth=0.25)
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history.history['acc'], color='b', label="Training accuracy")
ax[1].plot(history.history['val_acc'], color='r',label="Validation accuracy")
ax[1].grid(color='black', linestyle='-', linewidth=0.25)
legend = ax[1].legend(loc='best', shadow=True)

In [ ]:
score = keras_model.evaluate(x_test, y_test, verbose = 0)

In [ ]:
print('Test score:', score[0])
print('Test accuracy:', score[1])

In [ ]:
y_pred = keras_model.predict(x_test)

In [ ]:
y_pred = keras_model.predict(x_test)

y_test_class = np.argmax(y_test, axis=1)
y_pred_class = np.argmax(y_pred, axis=1)

cm = confusion_matrix(y_test_class, y_pred_class)

class_names = ['Olumlu', 'Olumsuz', 'Nötr']

fig, ax = plot_confusion_matrix(conf_mat=cm,
                                show_normed=True,
                                class_names=class_names,
                                figsize =(6,6))